<a href="https://colab.research.google.com/github/rdsiese/MANDES/blob/main/2_Vanilla_Options.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pricing Vanilla Options

### (1) The Black-Scholes Formula

In what follows, we use the following parameters: $S_0 = 100$, $K = 100$, $r=0.05$, $\sigma=0.2$ and $T=1$.

In [ ]:
S0=100; K=100; r=0.05; sigma=0.2; T=1

The Black-Scholes formula requires the use of a cumulative distribution function (of a standard normal distribution). The simplest way to handle this is with the command **norm.cdf** included in the statistical package **scipy.stats**, which we need to import.

In [ ]:
import numpy as np
import scipy.stats as ss
from matplotlib import pyplot as plt

We need to define three Python functions. The first one gives a call's price, the second one computes a put's price, and the third one obtains the put's price form the put-call parity relationship. Recall that the Black-Scholes formula uses $d_1$ and $d_2$, which we compute separately to simplify the code. To find the options's prices for the give initial values, the last line of the code simply calls the functions.

Call option price:

In [ ]:
def BS_call(S0,K,r,sigma,T):
    d1 = (np.log(S0/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = (np.log(S0/K) + (r - sigma**2/2)*T)/(sigma*np.sqrt(T))
    call_price = S0 * ss.norm.cdf(d1) - K*np.exp(-r*T) * ss.norm.cdf(d2)
    return call_price

BS_call(S0,K,r,sigma,T)

Put option price:

In [ ]:

def BS_put(S0,K,r,sigma,T):
    d1 = (np.log(S0/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = (np.log(S0/K) + (r - sigma**2/2)*T)/(sigma*np.sqrt(T))
    call_price = K*np.exp(-r*T) * ss.norm.cdf(-d2) - S0 * ss.norm.cdf(-d1)
    return call_price

BS_put(S0,K,r,sigma,T)

Put price using put-call parity:

In [ ]:

def BSput_parity(S0,K,r,sigma,T):
    put_parity = BS_call(S0,K,r,sigma,T) - S0 + np.exp(-r*T)*K
    return put_parity

BSput_parity(S0,K,r,sigma,T)

### (2) Using Monte Carlo Simulation

In this section we use the same initial values:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

S0=100; K=100; r=0.05; sigma=0.2; T=1

We run $n$ trajectories (scenarios) of the stock price, compute the __payoff__ at maturity, calculate the mean of the payoffs, and discount the mean to present value.

In [ ]:
n = 1000
z = np.random.normal(0,1,n)
S = S0*np.exp( (r-sigma**2/2)*T + sigma*np.sqrt(T)*z )
payoff = np.maximum(S-K,0)
call_price = np.exp(-r*T)*np.mean(payoff)

print(f'price: {call_price:.5f}')

The code above does not give us information regarding how accurate the price is. For this, we need to compute the mean standard error.

In [ ]:
mse = np.std(payoff) / np.sqrt(n)

print(f'price: {call_price:<13.5f} MSE: {mse:<10.3f} MSE/price: {mse/call_price:.3f}')

With 1,000 scenarios, the call's price is not very precise (the ratio MSE/price is around 4.6%, which is high).

We now compute the option's price running an increasing number of scenarios, and compare with the Black-Scholes formula.

In [ ]:
for n in[100, 1000, 10000, 25000, 100000, 1000000]:
    z=np.random.normal(0,1,n)
    S=S0*np.exp( (r-sigma**2/2)*T + sigma*np.sqrt(T)*z )
    payoff=np.maximum(S-K,0)
    call_price = np.exp(-r*T)*np.mean(payoff)
    mse = np.std(payoff) / np.sqrt(n)

    print(f'price: {call_price:<13.5f} MSE: {mse:<10.3f} n: {n:<11} MSE/price: {mse/call_price:.3f}')

print('BS:    10.45058')